In [1]:
import pandas as pd
import utils
import numpy as np
import warnings
from tqdm import tqdm
from copairs.map import mean_average_precision, multilabel
import logging

c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#all_profiles = {
#    "ORF": "c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_ORF",
#    "CRISPR": "c:\\Users\\ssivagur\\Documents\\GitHub\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_CRISPR",
#}

retrieval_label = "corum_complex"
multi_label_col = "Metadata_corum_complex_list"
annotation_col = "Metadata_complexname"


batch_size = 20000
null_size = 20000
fdr = 0.05

pos_sameby = [f"{multi_label_col}"]
pos_diffby = []
neg_sameby = []
neg_diffby = [f"{multi_label_col}"]

### ORF

In [5]:
orf_profiles = pd.read_parquet('c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_ORF.parquet')
orf_PhenotypicActivity = pd.read_csv('c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\PhenotypicActivity_ORF.csv.gz', compression='gzip')

Read protein class annotations

In [23]:
orf_annotation_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/orf_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022",f"{annotation_col}"]]
    .dropna()
    .assign(col=lambda x: list(x[f"{annotation_col}"].str.split("|")))
    .rename(columns={"col": f"{multi_label_col}"})
)

In [34]:
df = orf_profiles
phenotypic_activity_df = orf_PhenotypicActivity.query("below_corrected_p==True")


In [26]:
df = df.merge(phenotypic_activity_df, on="Metadata_JCP2022", how="inner")

In [27]:
df = df.merge(orf_annotation_df, on="Metadata_JCP2022", how="inner")

In [28]:
consensus_df = utils.consensus(df, "Metadata_NCBI_Gene_ID")

metadata_df = utils.get_metadata(consensus_df)
feature_df = utils.get_featuredata(consensus_df)
feature_values = feature_df.values

result = multilabel.average_precision(
        metadata_df,
        feature_values,
        pos_sameby,
        pos_diffby,
        neg_sameby,
        neg_diffby,
        batch_size=batch_size,
        multilabel_col=multi_label_col,
    )

agg_result = mean_average_precision(
        result, pos_sameby, null_size, threshold=0.05, seed=12527
    )

    #agg_result["Modality"] = modality

    #map_df = pd.concat([map_df, agg_result], ignore_index=True)

    #map_df.to_parquet(f"output/{retrieval_label}_retrieval.parquet")

c:\Users\ssivagur\Documents\GitHub\2024_Chandrasekaran_Morphmap\03.retrieve-annotations\utils.py:104: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  profiles_df[feature_cols].groupby([group_by_feature]).median().reset_index()


In [30]:
agg_result.to_csv('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\mAP_Corum.csv')

### CRISPR

In [31]:
#These details are already available in the profiles hence I am not using them now 
#crispr_controls_df = pd.DataFrame(
#    {
#        "Metadata_JCP2022": ["JCP2022_805264", "JCP2022_800001", "JCP2022_800002"],
#        "Metadata_pert_type": ["poscon", "negcon", "negcon"],
#    },
#    index=[0, 1, 2],
#)

crispr_annotation_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", f"{annotation_col}"]]
    .dropna()
    .assign(col=lambda x: list(x[f"{annotation_col}"].str.split("|")))
    .rename(columns={"col": f"{multi_label_col}"})
)

In [32]:
crispr_df = pd.read_parquet('c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_crispr.parquet')
phenotypic_activity_crispr_df = pd.read_csv('c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\PhenotypicActivity_CRISPR.csv.gz')

In [35]:
phenotypic_activity_crispr_df = phenotypic_activity_crispr_df.query("below_corrected_p==True")

In [36]:
crispr_df = crispr_df.merge(phenotypic_activity_crispr_df, on="Metadata_JCP2022", how="inner")


In [37]:
crispr_df = crispr_df.merge(
            crispr_annotation_df,
            on="Metadata_JCP2022",
            how="inner",
        )

In [38]:
consensus_df_crispr = utils.consensus(crispr_df, "Metadata_NCBI_Gene_ID")

metadata_df_crispr = utils.get_metadata(consensus_df_crispr)
feature_df_crispr = utils.get_featuredata(consensus_df_crispr)
feature_values_crispr = feature_df_crispr.values

result_crispr = multilabel.average_precision(
        metadata_df_crispr,
        feature_values_crispr,
        pos_sameby,
        pos_diffby,
        neg_sameby,
        neg_diffby,
        batch_size=batch_size,
        multilabel_col=multi_label_col,
    )

agg_result_crispr = mean_average_precision(
        result_crispr, pos_sameby, null_size, threshold=0.05, seed=12527
    )

In [39]:
agg_result_crispr.to_csv('C:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\mAP_Corum_CRISPR.csv')

In [40]:
all_profiles = {
    "ORF": "c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_ORF.parquet",
    "CRISPR": "c:\\Users\\ssivagur\\Documents\\GitHub\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_CRISPR.parquet",
}


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ssivagur\AppData\Local\Temp\ipykernel_26604\401110882.py:3: SyntaxWarning: invalid escape sequence '\s'
  "CRISPR": "c:\\Users\\ssivagur\\Documents\\GitHub\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\profiles_CRISPR.parquet",


In [41]:
map_df = pd.DataFrame()

for modality in all_profiles:
    print(f"Modality: {modality}")
    parquet_file_name = f"profiles_{modality}.parquet"
    phenotypic_activity_file_name = f"PhenotypicActivity_{modality}.csv.gz"

    df = pd.read_parquet(f"c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\{parquet_file_name}")
    phenotypic_activity_df = pd.read_csv(
        f"c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\{phenotypic_activity_file_name}"
    ).query("below_corrected_p==True")
    df = df.merge(phenotypic_activity_df, on="Metadata_JCP2022", how="inner")

    if modality == "ORF":
        df = df.merge(
            orf_annotation_df,
            on="Metadata_JCP2022",
            how="inner",
        )
    elif modality == "CRISPR":
        df = df.merge(
            crispr_annotation_df,
            on="Metadata_JCP2022",
            how="inner",
        )


    consensus_df = utils.consensus(df, "Metadata_NCBI_Gene_ID")

    metadata_df = utils.get_metadata(consensus_df)
    feature_df = utils.get_featuredata(consensus_df)
    feature_values = feature_df.values

    result = multilabel.average_precision(
        metadata_df,
        feature_values,
        pos_sameby,
        pos_diffby,
        neg_sameby,
        neg_diffby,
        batch_size=batch_size,
        multilabel_col=multi_label_col,
    )

    agg_result = mean_average_precision(
        result, pos_sameby, null_size, threshold=0.05, seed=12527
    )

    agg_result["Modality"] = modality

    map_df = pd.concat([map_df, agg_result], ignore_index=True)

    map_df.to_parquet(f"output/{retrieval_label}_retrieval.parquet")

Modality: ORF


c:\Users\ssivagur\Documents\GitHub\2024_Chandrasekaran_Morphmap\03.retrieve-annotations\utils.py:104: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  profiles_df[feature_cols].groupby([group_by_feature]).median().reset_index()


Modality: CRISPR


In [42]:
map_df.to_parquet(f'c:\\Users\\ssivagur\\Documents\\GitHub\\ssivagur\\FeatureSpaceIntegration\\ORFValidation\\{retrieval_label}_retrieval.parquet')

In [ ]:
crispr_controls_df = pd.DataFrame(
    {
        "Metadata_JCP2022": ["JCP2022_805264", "JCP2022_800001", "JCP2022_800002"],
        "Metadata_pert_type": ["poscon", "negcon", "negcon"],
    },
    index=[0, 1, 2],
)

crispr_annotation_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", f"{annotation_col}", "Metadata_NCBI_Gene_ID"]]
    .merge(crispr_controls_df, on="Metadata_JCP2022", how="left")
    .fillna(value={"Metadata_pert_type": "trt"})
    .dropna()
    .assign(col=lambda x: list(x[f"{annotation_col}"].str.split("|")))
    .rename(columns={"col": f"{multi_label_col}"})
)